Ali Alperen Usta

090180323

# Question 1

For this question use the World Bank Data for Turkey for the following indicators. Use [wbgapi](https://pypi.org/project/wbgapi/) for getting the data.

* [Literacy rate, adult female (SE.ADT.LITR.FE.ZS)](https://data.worldbank.org/indicator/SE.ADT.LITR.FE.ZS)
* [Labor force, female (SL.TLF.TOTL.FE.ZS)](https://data.worldbank.org/indicator/SL.TLF.TOTL.FE.ZS)
* [Poverty headcount ratio at national poverty lines (SI.POV.NAHC)](https://data.worldbank.org/indicator/SI.POV.NAHC)
* [Current health expenditure per capita (SH.XPD.CHEX.PC.CD)](https://data.worldbank.org/indicator/SH.XPD.CHEX.PC.CD)
* [GDP per capita (NY.GDP.PCAP.CD)](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)
* [Mortality rate, under-5 (SH.DYN.MORT)](https://data.worldbank.org/indicator/SH.DYN.MORT)


Using the [statsmodels](https://www.statsmodels.org/stable/index.html) library write the best linear regression model using child mortality as the dependent variable while the rest are considered as independent variables. Pay particular attention to the fact that the order of the variables put into the model significantly impacts the performance of the model. Choose the best model by considering

* with the minimum number of variables and their interactions,
* with the optimal ordering of the independent variables and their interactions,
* $R^2$-score of the model,
* statistical significance of the model coefficients,
* ANOVA analysis of the model.


In [64]:
import pandas as pd
import wbgapi as wb
import numpy as np
import statsmodels.api as sm

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from statsmodels.formula.api import ols
from statsmodels.formula.api import logit
from statsmodels.api import Logit

from collections import Counter


In [65]:
literacy = pd.DataFrame(list(wb.data.fetch('SE.ADT.LITR.FE.ZS'))) 
literacy 

,value,series,economy,aggregate,time
0,NaN,SE.ADT.LITR.FE.ZS,ZWE,False,YR2021
1,NaN,SE.ADT.LITR.FE.ZS,ZWE,False,YR2020
2,NaN,SE.ADT.LITR.FE.ZS,ZWE,False,YR2019
3,NaN,SE.ADT.LITR.FE.ZS,ZWE,False,YR2018
4,NaN,SE.ADT.LITR.FE.ZS,ZWE,False,YR2017
...,...,...,...,...,...
16487,NaN,SE.ADT.LITR.FE.ZS,AFE,True,YR1964
16488,NaN,SE.ADT.LITR.FE.ZS,AFE,True,YR1963
16489,NaN,SE.ADT.LITR.FE.ZS,AFE,True,YR1962
16490,NaN,SE.ADT.LITR.FE.ZS,AFE,True,YR1961


In [66]:
labor = pd.DataFrame(list(wb.data.fetch('SL.TLF.TOTL.FE.ZS'))) 
labor 

,value,series,economy,aggregate,time
0,51.133973,SL.TLF.TOTL.FE.ZS,ZWE,False,YR2021
1,51.180432,SL.TLF.TOTL.FE.ZS,ZWE,False,YR2020
2,51.489814,SL.TLF.TOTL.FE.ZS,ZWE,False,YR2019
3,51.512789,SL.TLF.TOTL.FE.ZS,ZWE,False,YR2018
4,51.538855,SL.TLF.TOTL.FE.ZS,ZWE,False,YR2017
...,...,...,...,...,...
16487,NaN,SL.TLF.TOTL.FE.ZS,AFE,True,YR1964
16488,NaN,SL.TLF.TOTL.FE.ZS,AFE,True,YR1963
16489,NaN,SL.TLF.TOTL.FE.ZS,AFE,True,YR1962
16490,NaN,SL.TLF.TOTL.FE.ZS,AFE,True,YR1961


In [67]:
poverty = pd.DataFrame(list(wb.data.fetch('SI.POV.NAHC'))) 
poverty 

,value,series,economy,aggregate,time
0,NaN,SI.POV.NAHC,ZWE,False,YR2021
1,NaN,SI.POV.NAHC,ZWE,False,YR2020
2,38.3,SI.POV.NAHC,ZWE,False,YR2019
3,NaN,SI.POV.NAHC,ZWE,False,YR2018
4,30.4,SI.POV.NAHC,ZWE,False,YR2017
...,...,...,...,...,...
16487,NaN,SI.POV.NAHC,AFE,True,YR1964
16488,NaN,SI.POV.NAHC,AFE,True,YR1963
16489,NaN,SI.POV.NAHC,AFE,True,YR1962
16490,NaN,SI.POV.NAHC,AFE,True,YR1961


In [68]:
health = pd.DataFrame(list(wb.data.fetch('SH.XPD.CHEX.PC.CD'))) 
health

,value,series,economy,aggregate,time
0,NaN,SH.XPD.CHEX.PC.CD,ZWE,False,YR2021
1,NaN,SH.XPD.CHEX.PC.CD,ZWE,False,YR2020
2,103.028272,SH.XPD.CHEX.PC.CD,ZWE,False,YR2019
3,141.369308,SH.XPD.CHEX.PC.CD,ZWE,False,YR2018
4,115.646201,SH.XPD.CHEX.PC.CD,ZWE,False,YR2017
...,...,...,...,...,...
16487,NaN,SH.XPD.CHEX.PC.CD,AFE,True,YR1964
16488,NaN,SH.XPD.CHEX.PC.CD,AFE,True,YR1963
16489,NaN,SH.XPD.CHEX.PC.CD,AFE,True,YR1962
16490,NaN,SH.XPD.CHEX.PC.CD,AFE,True,YR1961


In [69]:
gdp = pd.DataFrame(list(wb.data.fetch('NY.GDP.PCAP.CD')))  
gdp                                        

,value,series,economy,aggregate,time
0,1737.173977,NY.GDP.PCAP.CD,ZWE,False,YR2021
1,1214.509820,NY.GDP.PCAP.CD,ZWE,False,YR2020
2,1316.740657,NY.GDP.PCAP.CD,ZWE,False,YR2019
3,1254.642265,NY.GDP.PCAP.CD,ZWE,False,YR2018
4,1235.189032,NY.GDP.PCAP.CD,ZWE,False,YR2017
...,...,...,...,...,...
16487,180.228774,NY.GDP.PCAP.CD,AFE,True,YR1964
16488,199.784916,NY.GDP.PCAP.CD,AFE,True,YR1963
16489,172.271022,NY.GDP.PCAP.CD,AFE,True,YR1962
16490,162.555968,NY.GDP.PCAP.CD,AFE,True,YR1961


In [70]:
mortality = pd.DataFrame(list(wb.data.fetch('SH.DYN.MORT'))) 
mortality

,value,series,economy,aggregate,time
0,NaN,SH.DYN.MORT,ZWE,False,YR2021
1,53.9,SH.DYN.MORT,ZWE,False,YR2020
2,54.2,SH.DYN.MORT,ZWE,False,YR2019
3,54.8,SH.DYN.MORT,ZWE,False,YR2018
4,57.0,SH.DYN.MORT,ZWE,False,YR2017
...,...,...,...,...,...
16487,NaN,SH.DYN.MORT,AFE,True,YR1964
16488,NaN,SH.DYN.MORT,AFE,True,YR1963
16489,NaN,SH.DYN.MORT,AFE,True,YR1962
16490,NaN,SH.DYN.MORT,AFE,True,YR1961


In [71]:
def extract(data,country,variable):
    tmp = data[['time','value']][data['economy']==country]
    tmp.index = tmp.time
    del tmp['time']
    tmp.columns = [[variable]]
    return tmp

In [72]:
def model(country):
    lit = extract(literacy,country,'literacy')
    lab = extract(labor,country,'labor')
    hth = extract(health,country,'healthy')
    pov = extract(poverty,country,'poverty')
    gin = extract(gdp,country,'gdp')
    mor = extract(mortality,country,'mortality')
    dataframe = lit.join([lab,hth,pov,gin,mor])
    dataframe.dropna(inplace=True)
    return dataframe

In [73]:
df = model("TUR")
X = df.iloc[:,0:5] 
XX = sm.add_constant(X) 
Y = df['mortality'] 
model = sm.OLS(Y,XX)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              mortality   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     169.6
Date:                Sun, 06 Nov 2022   Prob (F-statistic):           3.81e-07
Time:                        23:45:17   Log-Likelihood:                -7.8492
No. Observations:                  13   AIC:                             27.70
Df Residuals:                       7   BIC:                             31.09
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            69.8438      9.987      6.994

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [74]:
corr_df=pd.concat([Y,X],axis=1)

In [75]:
corr_df.corr()

,mortality,literacy,labor,healthy,poverty,gdp
mortality,1.000000,-0.976415,-0.989861,-0.173542,0.885762,-0.624924
literacy,-0.976415,1.000000,0.981657,0.240982,-0.849316,0.708098
labor,-0.989861,0.981657,1.000000,0.097192,-0.867518,0.591484
healthy,-0.173542,0.240982,0.097192,1.000000,-0.192544,0.786933
poverty,0.885762,-0.849316,-0.867518,-0.192544,1.000000,-0.642485
gdp,-0.624924,0.708098,0.591484,0.786933,-0.642485,1.000000


In [76]:
model = ols('mortality ~  literacy + labor + poverty + healthy + gdp ', data=df).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:              mortality   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     169.6
Date:                Sun, 06 Nov 2022   Prob (F-statistic):           3.81e-07
Time:                        23:45:17   Log-Likelihood:                -7.8492
No. Observations:                  13   AIC:                             27.70
Df Residuals:                       7   BIC:                             31.09
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     69.8438      9.987      6.994      0.0

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,df,sum_sq,mean_sq,F,PR(>F)
literacy,1.0,296.579278,296.579278,815.327082,1.661736e-08
labor,1.0,8.414644,8.414644,23.132725,1.944516e-03
poverty,1.0,0.940111,0.940111,2.584462,1.519524e-01
healthy,1.0,1.829951,1.829951,5.030724,5.981897e-02
gdp,1.0,0.769732,0.769732,2.116072,1.890863e-01
Residual,7.0,2.546285,0.363755,NaN,NaN


**We established a linear regression model in which we tried to calculate the mortality variable over the independent variables.**

**The R^2 value seems high enough, but we need to examine other parameters.**

**The p-value of literacy, poverty and gdp variables is greater than 0.05. These values have 0 in the confidence interval range. The change in these variables cannot be expected to cause a change in the dependent variable.**







**When we examine the Anova table, we want to minimize the sum_sq and mean_sq values. In the order above, the error values of the variable literacy seem too large.**

**After looking at the anova chart as well, we can try removing some variables based on p value and confidence interval.**

**Dependent variables have an inverse correlation with the mortality variable. The correlation of "literacy" and "labor" variables is 0.98. It may make sense for us to exclude one of these variables from the model because one is sufficient when estimating.**

**When the correlation of the "mortality" variable with other variables is examined, the variable with the highest absolute value of the correlation value is the "labor" variable. so it makes more sense to use this variable in the model. Also, the p value of literacy variable is not what we want. We will rebuild the model by removing the "Literacy" variable from the model.**

In [77]:
model = ols('mortality ~ labor + healthy  + poverty  + gdp', data=df).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:              mortality   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     242.2
Date:                Sun, 06 Nov 2022   Prob (F-statistic):           2.23e-08
Time:                        23:45:48   Log-Likelihood:                -7.8525
No. Observations:                  13   AIC:                             25.70
Df Residuals:                       8   BIC:                             28.53
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     70.1838      7.670      9.151      0.0

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,df,sum_sq,mean_sq,F,PR(>F)
labor,1.0,304.804040,304.804040,957.157240,1.295003e-09
healthy,1.0,1.878245,1.878245,5.898137,4.129181e-02
poverty,1.0,0.456320,0.456320,1.432955,2.655482e-01
gdp,1.0,1.393817,1.393817,4.376917,6.978890e-02
Residual,8.0,2.547578,0.318447,NaN,NaN


In [78]:
model = ols('mortality ~  labor +  healthy', data=df).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:              mortality   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     348.7
Date:                Sun, 06 Nov 2022   Prob (F-statistic):           5.65e-10
Time:                        23:45:48   Log-Likelihood:                -11.401
No. Observations:                  13   AIC:                             28.80
Df Residuals:                      10   BIC:                             30.50
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     78.2286      2.596     30.136      0.0

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,df,sum_sq,mean_sq,F,PR(>F)
labor,1.0,304.804040,304.804040,693.096404,1.440841e-10
healthy,1.0,1.878245,1.878245,4.270957,6.565373e-02
Residual,10.0,4.397715,0.439771,NaN,NaN


**Although the last working model does not have the highest r squared value, it is the best model due to the range of the independent variables' confidence interval and p value.**

# Question 2

For this question use Yahoo's Finance API for the following tickers:

* Gold futures (GC=F)
* Silver futures (SI=F)
* Copper futures (HG=F)
* Platinum futures (PL=F)

1. Write the best linear regression model that explains gold futures closing prices in terms of opening prices of gold, silver, copper, and platinum futures.
2. Repeat the same for silver, copper and platinum prices.
3. Compare the models you obtained in Steps 1 and 2. Which model is better? How do you decide? Explain.

In [79]:
import yfinance as yf

In [80]:
gold = yf.download("GC=F")
silver = yf.download("SI=F")
copper = yf.download("HG=F")
platinum = yf.download("PL=F")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [81]:
def mix(close,open1,open2,open3):
    df=pd.concat([close["Close"],close["Open"],open1["Close"],open1["Open"],open2["Close"],open2["Open"],open3["Close"],open3["Open"]],axis=1)
    df=df.dropna()
    return df

In [82]:
data=mix(gold,silver,copper,platinum)
columns= ["gold_close","gold_open","silver_close","silver_open","copper_close","copper_open","platinum_close","platinum_open"]
data.columns=columns

In [83]:
data

,gold_close,gold_open,silver_close,silver_open,copper_close,copper_open,platinum_close,platinum_open
Date,,,,,,,,
2000-08-30 00:00:00-04:00,273.899994,273.899994,4.930000,4.950000,0.8850,0.8790,591.400024,593.900024
2000-08-31 00:00:00-04:00,278.299988,274.799988,5.003000,4.920000,0.8850,0.8850,586.700012,589.000000
2000-09-01 00:00:00-04:00,277.000000,277.000000,5.004000,5.035000,0.8890,0.8780,595.299988,588.000000
2000-09-05 00:00:00-04:00,275.799988,275.799988,4.998000,4.990000,0.9060,0.8960,601.299988,602.000000
2000-09-06 00:00:00-04:00,274.200012,274.200012,4.983000,5.000000,0.9015,0.9050,611.099976,603.000000
...,...,...,...,...,...,...,...,...
2022-10-31 00:00:00-04:00,1635.900024,1641.800049,19.125000,19.200001,3.4135,3.4095,939.400024,939.400024
2022-11-01 00:00:00-04:00,1645.000000,1630.800049,19.673000,19.125000,3.5095,3.4945,959.799988,959.799988
2022-11-02 00:00:00-04:00,1645.699951,1650.800049,19.600000,19.780001,3.5055,3.4985,960.200012,960.200012


In [84]:
model_gold = ols('gold_close ~  gold_open + silver_open + copper_open + platinum_open', data=data).fit()
print(model_gold.summary())
sm.stats.anova_lm(model_gold)

                            OLS Regression Results                            
Dep. Variable:             gold_close   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.509e+06
Date:                Sun, 06 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:48:06   Log-Likelihood:                -18732.
No. Observations:                4864   AIC:                         3.747e+04
Df Residuals:                    4859   BIC:                         3.751e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.4651      0.661      0.704

,df,sum_sq,mean_sq,F,PR(>F)
gold_open,1.0,1.301888e+09,1.301888e+09,1.003569e+07,0.000000
silver_open,1.0,1.012435e-02,1.012435e-02,7.804415e-05,0.992952
copper_open,1.0,2.903872e+02,2.903872e+02,2.238468e+00,0.134679
platinum_open,1.0,9.716272e+01,9.716272e+01,7.489850e-01,0.386841
Residual,4859.0,6.303380e+05,1.297259e+02,NaN,NaN


## Model Results

**-We are trying to predict the dependent variable of gold_close with 4 independent variables in our model.**

**-Firstly, we have to assess R-squared. R-squared shows how well the data fit the regression model. In the model output, the value of R squared is 1. Although it seems like a good result, it is not usual for this value to be 1 in regression models. In the next step, we will examine the correlation of variables in the data.**

**-In the data, it is not correct to interpret the intercept parameter as the opening values(silver_open, copper_open etc) cannot be 0.**

**-The coefficient size shows the effect of a change in that variable on the dependent variable. An important parameter to scale. While estimating the gold_close variable, the largest coefficient appears in the gold_open variable.**

**-Since the p>t value is lower than the significant level for gold_open, this arguments provide enough evidence to reject the null hypothesis. Changes in this independent variable associated with changes in the dependent variable.**

**-Since all variables except gold_open have a value of 0 in the confidence interval, we can say that the independent variables of this model are insufficient for estimation.**

## Anova Table 

**-Since the p>F value of the gold_open variable is smaller than 0.05, we can assume that variables other than gold_open have no effect on the estimation.**

In [85]:
data.corr()

,gold_close,gold_open,silver_close,silver_open,copper_close,copper_open,platinum_close,platinum_open
gold_close,1.000000,0.999758,0.869954,0.869799,0.829519,0.830044,0.500981,0.501384
gold_open,0.999758,1.000000,0.869686,0.870009,0.829315,0.830014,0.500949,0.501485
silver_close,0.869954,0.869686,1.000000,0.999327,0.854113,0.854688,0.761270,0.761452
silver_open,0.869799,0.870009,0.999327,1.000000,0.853636,0.854584,0.760743,0.761075
copper_close,0.829519,0.829315,0.854113,0.853636,1.000000,0.999405,0.743787,0.744005
copper_open,0.830044,0.830014,0.854688,0.854584,0.999405,1.000000,0.743762,0.744133
platinum_close,0.500981,0.500949,0.761270,0.760743,0.743787,0.743762,1.000000,0.999706
platinum_open,0.501384,0.501485,0.761452,0.761075,0.744005,0.744133,0.999706,1.000000


**When we examine the correlation table, the opening and closing values of the tickers are close to 1. This may be why the R squared value is 1 in the top line of code. We can omit gold_open in our estimation. Likewise, the variable silver_open may not be used for silver_close.**

**The correlation order for gold_close is as follows;
silver_open - copper_open - platinum_open**

In [86]:
model_gold1 = ols('gold_close ~  silver_open  + copper_open  +  platinum_open ', data=data).fit()
print(model_gold1.summary())
sm.stats.anova_lm(model_gold1)

                            OLS Regression Results                            
Dep. Variable:             gold_close   R-squared:                       0.878
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                 1.168e+04
Date:                Sun, 06 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:48:56   Log-Likelihood:                -32177.
No. Observations:                4864   AIC:                         6.436e+04
Df Residuals:                    4860   BIC:                         6.439e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       431.4906      8.454     51.040

,df,sum_sq,mean_sq,F,PR(>F)
silver_open,1.0,9.854214e+08,9.854214e+08,30178.878183,0.000000e+00
copper_open,1.0,3.632826e+07,3.632826e+07,1112.565914,7.553537e-220
platinum_open,1.0,1.220770e+08,1.220770e+08,3738.651614,0.000000e+00
Residual,4860.0,1.586920e+08,3.265269e+04,NaN,NaN


**After removing the gold_open variable from the model, the p-value of all variables became 0. Also, R squared is 0.878.**

In [87]:
model_silver2 = ols('silver_close ~  gold_open + copper_open  +  platinum_open ', data=data).fit()
print(model_silver2.summary())
sm.stats.anova_lm(model_silver2)

                            OLS Regression Results                            
Dep. Variable:           silver_close   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                 1.431e+04
Date:                Sun, 06 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:48:57   Log-Likelihood:                -11841.
No. Observations:                4864   AIC:                         2.369e+04
Df Residuals:                    4860   BIC:                         2.372e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -6.5910      0.129    -51.001

,df,sum_sq,mean_sq,F,PR(>F)
gold_open,1.0,275715.317303,275715.317303,36144.808684,0.0
copper_open,1.0,20677.591958,20677.591958,2710.722105,0.0
platinum_open,1.0,31066.699430,31066.699430,4072.678727,0.0
Residual,4860.0,37072.445280,7.628075,NaN,NaN


In [88]:
model_copper = ols('copper_close ~  gold_open + silver_open  +  platinum_open ', data=data).fit()
print(model_copper.summary())
sm.stats.anova_lm(model_copper)

                            OLS Regression Results                            
Dep. Variable:           copper_close   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     8059.
Date:                Sun, 06 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:48:58   Log-Likelihood:                -3072.9
No. Observations:                4864   AIC:                             6154.
Df Residuals:                    4860   BIC:                             6180.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.3247      0.026    -12.492

,df,sum_sq,mean_sq,F,PR(>F)
gold_open,1.0,4140.017827,4140.017827,19970.068333,0.0
silver_open,1.0,432.284161,432.284161,2085.194942,0.0
platinum_open,1.0,439.697523,439.697523,2120.954533,0.0
Residual,4860.0,1007.532188,0.207311,NaN,NaN


In [89]:
model_platinum = ols('platinum_close ~  gold_open + silver_open  +  copper_open ', data=data).fit()
print(model_platinum.summary())
sm.stats.anova_lm(model_platinum)

                            OLS Regression Results                            
Dep. Variable:         platinum_close   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     5763.
Date:                Sun, 06 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:48:58   Log-Likelihood:                -31926.
No. Observations:                4864   AIC:                         6.386e+04
Df Residuals:                    4860   BIC:                         6.389e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     524.4287      6.521     80.418      

,df,sum_sq,mean_sq,F,PR(>F)
gold_open,1.0,1.636960e+08,1.636960e+08,5558.105956,0.0
silver_open,1.0,2.832884e+08,2.832884e+08,9618.727845,0.0
copper_open,1.0,6.218676e+07,6.218676e+07,2111.479070,0.0
Residual,4860.0,1.431355e+08,2.945175e+04,NaN,NaN


**After running all models, the model with the highest R squared value; It is the model with silver_close. the R squared value of the model is 0.898.**

# Question 3

1. Write a function that takes a ticker symbol and returns a pandas dataframe that for each day puts a 1 when the closing price is higher than the opening price, a 0 when the closing price is lower than the opening price.
2. Write the best logistic regression that predicts the time series you obtain from Step 1 for gold futures against the opening prices of gold, silver, copper, and platinum prices.
3. Repeat the same for silver, copper, and platinum prices.
4. Compare the models you obtained from Steps 2 and 3. Decide which is the best model, and explain your reasoning.
5. Does any of the models provide a good fit? Explain.

In [90]:
gold_2=gold.copy()
silver_2=silver.copy()
copper_2=copper.copy()
platinum_2=platinum.copy()

In [91]:
def label(data,name):
    data[name]=data["Close"] > data["Open"]
    data[name] = data[name].astype("int32").replace({True:1 , False:0})
    return data

**In the function, values where the closing value is greater than the opening value are set as boolean. These values are replaced by 1 and 0 with the replace function.**

In [92]:
gold_2=label(gold_2,"gold_label")
silver_2=label(silver_2,"silver_label")
copper_2=label(copper_2,"copper_label")
platinum_2=label(platinum_2,"platinum_label")

In [93]:
gold_2

,Open,High,Low,Close,Adj Close,Volume,gold_label
Date,,,,,,,
2000-08-30 00:00:00-04:00,273.899994,273.899994,273.899994,273.899994,273.899994,0,0
2000-08-31 00:00:00-04:00,274.799988,278.299988,274.799988,278.299988,278.299988,0,1
2000-09-01 00:00:00-04:00,277.000000,277.000000,277.000000,277.000000,277.000000,0,0
2000-09-05 00:00:00-04:00,275.799988,275.799988,275.799988,275.799988,275.799988,2,0
2000-09-06 00:00:00-04:00,274.200012,274.200012,274.200012,274.200012,274.200012,0,0
...,...,...,...,...,...,...,...
2022-10-31 00:00:00-04:00,1641.800049,1641.800049,1633.400024,1635.900024,1635.900024,522,0
2022-11-01 00:00:00-04:00,1630.800049,1653.699951,1630.800049,1645.000000,1645.000000,985,1
2022-11-02 00:00:00-04:00,1650.800049,1664.699951,1634.000000,1645.699951,1645.699951,612,0


In [94]:
df2=pd.concat([data,gold_2["gold_label"],silver_2["silver_label"],copper_2["copper_label"],platinum_2["platinum_label"]],axis=1)

In [95]:
df2.dropna()

,gold_close,gold_open,silver_close,silver_open,copper_close,copper_open,platinum_close,platinum_open,gold_label,silver_label,copper_label,platinum_label
Date,,,,,,,,,,,,
2000-08-30 00:00:00-04:00,273.899994,273.899994,4.930000,4.950000,0.8850,0.8790,591.400024,593.900024,0.0,0.0,1.0,0.0
2000-08-31 00:00:00-04:00,278.299988,274.799988,5.003000,4.920000,0.8850,0.8850,586.700012,589.000000,1.0,1.0,0.0,0.0
2000-09-01 00:00:00-04:00,277.000000,277.000000,5.004000,5.035000,0.8890,0.8780,595.299988,588.000000,0.0,0.0,1.0,1.0
2000-09-05 00:00:00-04:00,275.799988,275.799988,4.998000,4.990000,0.9060,0.8960,601.299988,602.000000,0.0,1.0,1.0,0.0
2000-09-06 00:00:00-04:00,274.200012,274.200012,4.983000,5.000000,0.9015,0.9050,611.099976,603.000000,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31 00:00:00-04:00,1635.900024,1641.800049,19.125000,19.200001,3.4135,3.4095,939.400024,939.400024,0.0,0.0,1.0,0.0
2022-11-01 00:00:00-04:00,1645.000000,1630.800049,19.673000,19.125000,3.5095,3.4945,959.799988,959.799988,1.0,1.0,1.0,0.0
2022-11-02 00:00:00-04:00,1645.699951,1650.800049,19.600000,19.780001,3.5055,3.4985,960.200012,960.200012,0.0,0.0,1.0,0.0


In [96]:
model_gold = logit('gold_label ~ gold_open + silver_open + copper_open + platinum_open', data=df2).fit()
model_gold.summary()

Optimization terminated successfully.
         Current function value: 0.673709
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             gold_label   No. Observations:                 4864
Model:                          Logit   Df Residuals:                     4859
Method:                           MLE   Df Model:                            4
Date:                Sun, 06 Nov 2022   Pseudo R-squ.:                 0.01211
Time:                        23:49:02   Log-Likelihood:                -3276.9
converged:                       True   LL-Null:                       -3317.1
Covariance Type:            nonrobust   LLR p-value:                 1.487e-16
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.7144      0.120     -5.933      0.000      -0.950      -0.478
gold_open      9.671e-05      0.000      0.599      0.549      -0.000       0.000
silver_open       0.0280      0.011      2.656      0.008       0.007       0.049
copper_open       0.0340      0.064      0.530      0.596      -0.092       0.160
platinum_open    -0.0002      0.000     -1.416      0.157      -0.001    9.25e-05
=================================================================================
"""

In [97]:
model_silver = logit('silver_label ~ gold_open  + silver_open + copper_open + platinum_open', data=df2).fit()
model_silver.summary()

Optimization terminated successfully.
         Current function value: 0.631084
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           silver_label   No. Observations:                 4864
Model:                          Logit   Df Residuals:                     4859
Method:                           MLE   Df Model:                            4
Date:                Sun, 06 Nov 2022   Pseudo R-squ.:                 0.01346
Time:                        23:49:02   Log-Likelihood:                -3069.6
converged:                       True   LL-Null:                       -3111.5
Covariance Type:            nonrobust   LLR p-value:                 2.762e-17
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.9790      0.128     -7.650      0.000      -1.230      -0.728
gold_open        -0.0001      0.000     -0.627      0.531      -0.000       0.000
silver_open       0.0451      0.011      4.164      0.000       0.024       0.066
copper_open       0.0358      0.067      0.537      0.591      -0.095       0.166
platinum_open    -0.0004      0.000     -2.292      0.022      -0.001   -5.92e-05
=================================================================================
"""

In [98]:
model_copper = logit('copper_label ~ gold_open + silver_open + copper_open + platinum_open', data=df2).fit()
model_copper.summary()

Optimization terminated successfully.
         Current function value: 0.692917
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           copper_label   No. Observations:                 4864
Model:                          Logit   Df Residuals:                     4859
Method:                           MLE   Df Model:                            4
Date:                Sun, 06 Nov 2022   Pseudo R-squ.:               5.082e-05
Time:                        23:49:03   Log-Likelihood:                -3370.3
converged:                       True   LL-Null:                       -3370.5
Covariance Type:            nonrobust   LLR p-value:                    0.9869
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0106      0.116      0.092      0.927      -0.217       0.238
gold_open     -1.006e-05      0.000     -0.063      0.950      -0.000       0.000
silver_open       0.0032      0.010      0.309      0.757      -0.017       0.024
copper_open      -0.0162      0.063     -0.256      0.798      -0.140       0.108
platinum_open -4.589e-05      0.000     -0.274      0.784      -0.000       0.000
=================================================================================
"""

In [99]:
model_platinum = logit('platinum_label ~ gold_open + silver_open + copper_open + platinum_open', data=df2).fit()
model_platinum.summary()

Optimization terminated successfully.
         Current function value: 0.533813
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         platinum_label   No. Observations:                 4864
Model:                          Logit   Df Residuals:                     4859
Method:                           MLE   Df Model:                            4
Date:                Sun, 06 Nov 2022   Pseudo R-squ.:                 0.07848
Time:                        23:49:03   Log-Likelihood:                -2596.5
converged:                       True   LL-Null:                       -2817.6
Covariance Type:            nonrobust   LLR p-value:                 2.042e-94
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.8905      0.131      6.783      0.000       0.633       1.148
gold_open        -0.0018      0.000     -9.321      0.000      -0.002      -0.001
silver_open       0.0656      0.013      5.140      0.000       0.041       0.091
copper_open      -0.0437      0.078     -0.559      0.576      -0.197       0.110
platinum_open    -0.0009      0.000     -4.229      0.000      -0.001      -0.000
=================================================================================
"""

**When we examine the 4 models, we can see that the r squared values of the models are very low. It can be said that none of the models provide good fit.**

# Question 4

For this question use the following [data](https://archive.ics.uci.edu/ml/datasets/credit+approval):


In [100]:
credit = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header=None)

fn = {'+': 1, '-': 0}

X = credit.replace('?',0).iloc[:,[1,2,7,10,14]]
y = credit.iloc[:,15].map(lambda x: fn.get(x,0))

1. Split the data into training and test set.
2. Write different logistic regression models predicting y against X.
3. Construct [confusion matrices](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) on the test data set for these different models.
4. Analyze these models. Explain which model is the best model you have found.
5. Repeat Steps 1-4 several times. Does your best model stay as the best model? What should be the correct protocol to decide on the best model explaining the data?

In [101]:
Counter(y)

Counter({1: 307, 0: 383})

**It can be said that the dependent variable is proportionally distributed.**

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)

#### Logistic Regression

In [103]:
logisticRegr = LogisticRegression(max_iter=2000,solver = 'liblinear')
logisticRegr.fit(X_train,y_train)
y_pred=logisticRegr.predict(X_test)
logisticRegr.score(X_test, y_test)


0.7630057803468208

In [104]:
y_pred = logisticRegr.predict(X_test)
confusion_matrix(y_test,y_pred)

array([[91,  9],
       [32, 41]], dtype=int64)

**The percentage of the model predicting 0 is greater than the percentage of predicting 1.**

#### Support Vector Machine 

In [105]:
svc_model=SVC(max_iter=2000, kernel="rbf" )
svc_model.fit(X_train,y_train)
y_pred_svc=svc_model.predict(X_test)
svc_model.score(X_test,y_test)


0.653179190751445

In [106]:
confusion_matrix(y_test,y_pred_svc)

array([[93,  7],
       [53, 20]], dtype=int64)

In [107]:
accuracy_score(y_test,y_pred_svc)

0.653179190751445

**In the model we built with the SVM algorithm, it seems that 0 values are predicted successfully, while 1 values are mostly predicted incorrectly.**

**Once we run the models, Logistic regression is more successful than SVM. For the task requested in step 5, we will write a function and try to run the models 100 times.**

In [112]:
def repeat_model_logistic(X,y):
    list1 = []
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)
        model = LogisticRegression(max_iter=2000)
        model.fit(X_train,y_train)
        list1.append(model.score(X_test,y_test))
    return (min(list1),max(list1))

In [113]:
repeat_model_logistic(X,y)

(0.6878612716763006, 0.8265895953757225)

In [114]:
def repeat_model_SVM(X,y):
    list2 = []
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)
        model = SVC(max_iter=2000, kernel="rbf" )
        model.fit(X_train,y_train)
        list2.append(model.score(X_test,y_test))
    return (min(list2),max(list2))

In [115]:
repeat_model_SVM(X,y)

(0.5895953757225434, 0.7514450867052023)

**After running both functions, it seems to give better score for Logistic regression for this data.**